In [3]:
from evaluate import *

In [6]:
base_folder = "Output/Reproduce"

# 4.2.1

In [8]:
for loss in ["MSELoss", "AreaWeightedMSELoss"]:
    for use_coord_conv in [True, False]:
        for use_cylindrical_padding in [True, False]:
            conditions = {
                "DATASET_DESCRIPTION": {"PREDICTOR_VARIABLES" : {"temp": ["temp_1"],
                                                                 "precip": ["precip"]},
                                        "PRECIP_WEIGHTING" : False
                                       },
                "MODEL_TRAINING_DESCRIPTION": {"MODEL_TYPE": "UNet_Flat",
                                              "USE_COORD_CONV": use_coord_conv,
                                              "USE_CYLINDRICAL_PADDING": use_cylindrical_padding,
                                              "LOSS": loss,
                                              }
            }
            descriptions, predictions, gt, masks = load_data_for_comparison(base_folder, conditions)
            r2 = np.zeros(len(predictions))
            for i in range(len(predictions)):
                r2[i] = get_weighted_average(get_r2(predictions[i], gt[i]), descriptions[i]["DATASET_DESCRIPTION"])
            print('Loss: {}, CoordConv: {}, CylindricalPadding: {}, R2 = {:.3f} +/- {:.3f}'.format(loss, use_coord_conv, use_cylindrical_padding, np.mean(r2), np.std(r2)))

10 matching runs found
Loss: MSELoss, CoordConv: True, CylindricalPadding: True, R2 = 0.232 +/- 0.004
10 matching runs found
Loss: MSELoss, CoordConv: True, CylindricalPadding: False, R2 = 0.230 +/- 0.004
10 matching runs found
Loss: MSELoss, CoordConv: False, CylindricalPadding: True, R2 = 0.224 +/- 0.006
10 matching runs found
Loss: MSELoss, CoordConv: False, CylindricalPadding: False, R2 = 0.223 +/- 0.004
10 matching runs found
Loss: AreaWeightedMSELoss, CoordConv: True, CylindricalPadding: True, R2 = 0.236 +/- 0.003
10 matching runs found
Loss: AreaWeightedMSELoss, CoordConv: True, CylindricalPadding: False, R2 = 0.234 +/- 0.004
10 matching runs found
Loss: AreaWeightedMSELoss, CoordConv: False, CylindricalPadding: True, R2 = 0.230 +/- 0.004
10 matching runs found
Loss: AreaWeightedMSELoss, CoordConv: False, CylindricalPadding: False, R2 = 0.227 +/- 0.002


# 4.2.2

TODO Interpolations: Requires cdo.

Flat grid:

In [14]:
linreg_baseline_c = {
    "DATASET_DESCRIPTION": {},
    "MODEL_TRAINING_DESCRIPTION": {"MODEL_TYPE": "LinReg_Pixelwise"}
}
randomforest_baseline_c = {
    "DATASET_DESCRIPTION": {},
    "MODEL_TRAINING_DESCRIPTION": {"MODEL_TYPE": "RandomForest_Pixelwise"}
}

pca_baseline_c = {
    "DATASET_DESCRIPTION": {},
    "MODEL_TRAINING_DESCRIPTION": {"MODEL_TYPE": "PCA_Flat",
                                  "REGTYPE": "linreg"}
}

unet_unmodified_c = {
    "DATASET_DESCRIPTION": {"PREDICTOR_VARIABLES" : {"temp": ["temp_1"],
                                                                 "precip": ["precip"]},
                                        "PRECIP_WEIGHTING" : False
                                       },
    "MODEL_TRAINING_DESCRIPTION": {"MODEL_TYPE": "UNet_Flat",
                                  "USE_COORD_CONV": False,
                                  "USE_CYLINDRICAL_PADDING": False,
                                  "LOSS": "MSELoss"}
}

unet_modified_c = {
    "DATASET_DESCRIPTION": {"PREDICTOR_VARIABLES" : {"temp": ["temp_1"],
                                                                 "precip": ["precip"]},
                                        "PRECIP_WEIGHTING" : False
                                       },
    "MODEL_TRAINING_DESCRIPTION": {"MODEL_TYPE": "UNet_Flat",
                                  "USE_COORD_CONV": True,
                                  "USE_CYLINDRICAL_PADDING": True,
                                  "LOSS": "AreaWeightedMSELoss"}
}

In [16]:
descriptions_u_u, predictions_u_u, gt_u_u, masks_u_u = load_data_for_comparison(base_folder, unet_unmodified_c)
descriptions_m_u, predictions_m_u, gt_m_u, masks_m_u = load_data_for_comparison(base_folder, unet_modified_c)
descriptions_rf, predictions_rf, gt_rf, masks_rf = load_data_for_comparison(base_folder, randomforest_baseline_c)
descriptions_lr, predictions_lr, gt_lr, masks_lr = load_data_for_comparison(base_folder, linreg_baseline_c)
descriptions_pca, predictions_pca, gt_pca, masks_pca = load_data_for_comparison(base_folder, pca_baseline_c)

r2_u_u = np.zeros(len(predictions_u_u))
r2_m_u = np.zeros(len(predictions_m_u))
r2_rf = np.zeros(len(predictions_rf))
r2_lr = np.zeros(len(predictions_lr))
r2_pca = np.zeros(len(predictions_pca))

for i in range(len(predictions_u_u)):
    r2_u_u[i] = get_weighted_average(get_r2(predictions_u_u[i], gt_u_u[i]), descriptions_u_u[i]["DATASET_DESCRIPTION"])

for i in range(len(predictions_m_u)):
    r2_m_u[i] = get_weighted_average(get_r2(predictions_m_u[i], gt_m_u[i]), descriptions_m_u[i]["DATASET_DESCRIPTION"])
    
for i in range(len(predictions_rf)):
    r2_rf[i] = get_weighted_average(get_r2(predictions_rf[i], gt_rf[i]), descriptions_rf[i]["DATASET_DESCRIPTION"])
    
for i in range(len(predictions_lr)):
    r2_lr[i] = get_weighted_average(get_r2(predictions_lr[i], gt_lr[i]), descriptions_lr[i]["DATASET_DESCRIPTION"])
    
for i in range(len(predictions_pca)):
    r2_pca[i] = get_weighted_average(get_r2(predictions_pca[i], gt_pca[i]), descriptions_pca[i]["DATASET_DESCRIPTION"])
    
print('Unmodified UNet: R2 = {:.3f} \pm {:.3f}'.format(np.mean(r2_u_u), np.std(r2_u_u)))
print('Modified UNet: R2 = {:.3f} \pm {:.3f}'.format(np.mean(r2_m_u), np.std(r2_m_u)))
print('Random Forest: R2 = {:.3f} \pm {:.3f}'.format(np.mean(r2_rf), np.std(r2_rf)))
print('Pixelwise Linear regression: R2 = {:.3f} \pm {:.3f}'.format(np.mean(r2_lr), np.std(r2_lr)))
print('PCA Regression: R2 = {:.3f} \pm {:.3f}'.format(np.mean(r2_pca), np.std(r2_pca)))

10 matching runs found
10 matching runs found
1 matching runs found
1 matching runs found
1 matching runs found
Unmodified UNet: R2 = 0.223 \pm 0.004
Modified UNet: R2 = 0.236 \pm 0.003
Random Forest: R2 = 0.139 \pm 0.000
Pixelwise Linear regression: R2 = 0.169 \pm 0.000
PCA Regression: R2 = 0.189 \pm 0.000


icosahedral grid:

In [7]:
ico_baseline_c = {
    "DATASET_DESCRIPTION": {},
    "MODEL_TRAINING_DESCRIPTION": {"MODEL_TYPE": "PCA_Ico",
                                  "REGTYPE": "linreg"}
}

ico_unet_c = {
    "DATASET_DESCRIPTION": {"PREDICTOR_VARIABLES" : {"temp": ["temp_1"],
                                                                 "precip": ["precip"]},
                                        "PRECIP_WEIGHTING" : False
                                       },
    "MODEL_TRAINING_DESCRIPTION": {"MODEL_TYPE": "UNet_Ico"}
}

In [8]:
descriptions_ico_unet, predictions_ico_unet, gt_ico_unet, masks_ico_unet = load_data_for_comparison(base_folder, ico_unet_c)
descriptions_ico_bl, predictions_ico_bl, gt_ico_bl, masks_ico_bl = load_data_for_comparison(base_folder, ico_baseline_c)

r2_ico_unet = np.zeros(len(predictions_ico_unet))
r2_ico_bl = np.zeros(len(predictions_ico_bl))

for i in range(len(predictions_ico_unet)):
    r2_ico_unet[i] = np.mean(get_r2(predictions_ico_unet[i], gt_ico_unet[i]), axis=(1,2))

for i in range(len(predictions_ico_bl)):
    r2_ico_bl[i] = np.mean(get_r2(predictions_ico_bl[i], gt_ico_bl[i]), axis=(1,2))
    
print('Ico Unet: R2 = {:.3f} \pm {:.3f}'.format(np.mean(r2_ico_unet), np.std(r2_ico_unet)))
print('Ico Baseline: R2 = {:.3f} \pm {:.3f}'.format(np.mean(r2_ico_bl), np.std(r2_ico_bl)))

10 matching runs found
1 matching runs found
Ico Unet: R2 = 0.250 \pm 0.006
Ico Baseline: R2 = 0.213 \pm 0.000
